In [32]:
import pandas as pd
import numpy as np
import os
import json

pd.set_option('display.max_columns', 500)

In [153]:
'''
-read in the 2.Indices_Target.csv file
-filter on the NA-s in the snapshot_target column
-get the unique target nums
'''

filename='2.Indices_Target.csv'
df=pd.read_csv(f'C:/Users/511232/Desktop/Scorecard_coxcomb/{filename}', dtype='str')

#filter out the dataframe where snapshot_target is NA
df_na=df[df['snapshot_target'].isna()].copy()
target_num_NA=set(df_na['Target_number'])

In [157]:
'''
Code	Color
0	    Grey
1	    Red
2	    Yellow
3	    Green

-the scorecard.xlsx file, filter on color ('Color Scheme'=0) & ('Gap'=blank) & ('target' in target_num_NA)
-groupby ['geoareaname', 'target'] and apply a function that will take the average of 'Trend'
-and standardize the result of the average to be between [-10;+10]
formula=N= -10 + [(A+100)/10]

-generate coxcomb color using the following benchmarks:
if N is negative --> red, takes the value of 1
if N is postive but less or equal to 5.33 --> yellow, takes the value of 2
if N is greater than 5.33--> green, takes the value of 3"

'''

filename='scorecard.xlsx'
#read in the scorecard.xlsx
df=pd.read_excel('C:/Users/511232/Desktop/Scorecard_coxcomb/scorecard.xlsx', dtype=str)
df['Trend'] = df['Trend'].astype(float)

#filter color scheme=0 & Gap=blank
df_filtered=df[(df['Color Scheme']=='0') & (df['Gap'].isna()) & (df['target'].isin(target_num_NA))].copy()


def process(d):
    '''will calculate average trend and standardize the value'''
    trend_avg=d['Trend'].mean()
    trend_avg_standardized=-10+((trend_avg+100)/10)
    return(trend_avg_standardized)

#apply the function on the df_filtered
standardized_df=pd.DataFrame(df_filtered.groupby(['geoareaname','target'],group_keys=False).apply(process))
standardized_df.reset_index(drop=False, inplace=True)
standardized_df.columns=['geoareaname','target','standardized_value']



In [112]:
#merge with the original file scorecard.xlsx
result=pd.merge(df,standardized_df, how='left', on=['geoareaname','target'])
result.to_excel('processed_file.xlsx',index=False)

In [119]:
df=pd.read_excel('C:/Users/511232/Desktop/Scorecard_coxcomb/scorecard.xlsx')
